ETL with Spark on EMR

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType

import pyspark.sql.functions as F

In [ ]:
bucket = "s3://zkan-dataset"
landing_zone = f"{bucket}/landing"
cleaned_zone_csv = f"{bucket}/cleaned"
cleaned_zone_parquet = f"{bucket}/cleaned_parquet"

In [ ]:
spark = SparkSession.builder \
    .appName("ETL") \
    .getOrCreate()

In [ ]:
data = spark.read.option("multiline", "true").json(landing_zone)

In [ ]:
data.printSchema()

In [ ]:
data.createOrReplaceTempView("staging_events")

In [ ]:
table = spark.sql("""
    select
        id
        , type
        , created_at
        , to_date(created_at) as date
        , year(created_at) as year
        
    from
        staging_events
""")

In [ ]:
table.show()

In [ ]:
table.write.partitionBy("year").mode("overwrite").csv(cleaned_zone)

In [ ]:
table.write.partitionBy("year").mode("overwrite").parquet(cleaned_zone_parquet)